In [7]:
import time
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import cv2

In [8]:
### Data

class Dataset:
    def __init__(self, batch_size):
        self.batch_size = batch_size
        self.encodedPixel = pd.read_csv("train_ship_segmentations.csv", header=0)
        self.x_train = pd.read_csv("augmentation/train1.csv", header=-1)[1]
        self.x_test = pd.read_csv("augmentation/test1.csv", header=-1)[1]
        self.train_batch_count = self.x_train.shape[0] // self.batch_size
        self.test_batch_count = self.x_test.shape[0] // 10
        self.pp_mean = np.load("mean.npy")
        self.shuffle = np.random.permutation(self.x_train.shape[0])
    
    def normalize(self, batch_images):
        return (batch_images - self.pp_mean) / 128.0
    
    def next_aug_train_batch(self, idx):
        link_batch_images = self.x_train[self.shuffle[idx * self.batch_size: (idx + 1) * self.batch_size]]
        batch_images = []
        batch_mask = []
        for x in link_batch_images:
            img = cv2.imread('all/train/'+x)
            batch_images.append(img)
            
            rle = self.encodedPixel.query('ImageId=="'+x+'"')['EncodedPixels'].tolist()
            mask = self.masks_as_image(rle)
            batch_mask.append(mask)
        
        return self.normalize(np.array(batch_images)), np.array(batch_mask)
    
    def next_test_batch(self, idx):
        link_batch_images = self.x_test[self.shuffle[idx * 10: (idx + 1) * 10]]
        batch_images = []
        batch_mask = []
        for x in link_batch_images:
            img = cv2.imread('all/train/'+x)
            batch_images.append(img)
            
            rle = self.encodedPixel.query('ImageId=="'+x+'"')['EncodedPixels'].tolist()
            mask = self.masks_as_image(rle)
            batch_mask.append(mask)
        
        return self.normalize(np.array(batch_images)), np.array(batch_mask)
    
    
    def masks_as_image(self, in_mask_list, all_masks=None):
        if all_masks is None:
            all_masks = np.zeros((768, 768), dtype = np.int16)
        #if isinstance(in_mask_list, list):
        for mask in in_mask_list:
            if isinstance(mask, str):
                all_masks += self.rle_decode(mask)
        return np.expand_dims(all_masks, -1)

    def rle_decode(self, mask_rle, shape=(768, 768)):
        s = mask_rle.split()
        starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
        starts -= 1
        ends = starts + lengths
        img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
        for lo, hi in zip(starts, ends):
            img[lo:hi] = 1
        return img.reshape(shape).T  # Needed to align to RLE direction


In [9]:
def image_augmentation(image, mask):
    """Returns (maybe) augmented images
    (1) Random flip (left <--> right)
    (2) Random flip (up <--> down)
    (3) Random brightness
    (4) Random hue
    Args:
        image (3-D Tensor): Image tensor of (H, W, C)
        mask (3-D Tensor): Mask image tensor of (H, W, 1)
    Returns:
        image: Maybe augmented image (same shape as input `image`)
        mask: Maybe augmented mask (same shape as input `mask`)
    """
    concat_image = tf.concat([image, mask], axis=-1)

    maybe_flipped = tf.image.random_flip_left_right(concat_image)
    maybe_flipped = tf.image.random_flip_up_down(concat_image)

    image = maybe_flipped[:, :, :-1]
    mask = maybe_flipped[:, :, -1:]

    image = tf.image.random_brightness(image, 0.7)
    image = tf.image.random_hue(image, 0.3)

    return image, mask


def get_image_mask(queue, augmentation=False):
    """Returns `image` and `mask`
    Input pipeline:
        Queue -> CSV -> FileRead -> Decode JPEG
    (1) Queue contains a CSV filename
    (2) Text Reader opens the CSV
        CSV file contains two columns
        ["path/to/image.jpg", "path/to/mask.jpg"]
    (3) File Reader opens both files
    (4) Decode JPEG to tensors
    Notes:
        height, width = 640, 960
    Returns
        image (3-D Tensor): (640, 960, 3)
        mask (3-D Tensor): (640, 960, 1)
    """
    text_reader = tf.TextLineReader(skip_header_lines=1)
    _, csv_content = text_reader.read(queue)

    image_path, mask_path = tf.decode_csv(
        csv_content, record_defaults=[[""], [""]])

    image_file = tf.read_file(image_path)
    mask_file = tf.read_file(mask_path)

    image = tf.image.decode_jpeg(image_file, channels=3)
    image.set_shape([768, 768, 3])
    image = tf.cast(image, tf.float32)

    mask = tf.image.decode_jpeg(mask_file, channels=1)
    mask.set_shape([768, 768, 1])
    mask = tf.cast(mask, tf.float32)
    mask = mask / (tf.reduce_max(mask) + 1e-7)

    if augmentation:
        image, mask = image_augmentation(image, mask)

    return image, mask

In [14]:
def conv_conv_pool(input_,
                   n_filters,
                   training,
                   flags,
                   name,
                   pool=True,
                   activation=tf.nn.relu):
    """{Conv -> BN -> RELU}x2 -> {Pool, optional}
    Args:
        input_ (4-D Tensor): (batch_size, H, W, C)
        n_filters (list): number of filters [int, int]
        training (1-D Tensor): Boolean Tensor
        name (str): name postfix
        pool (bool): If True, MaxPool2D
        activation: Activaion functions
    Returns:
        net: output of the Convolution operations
        pool (optional): output of the max pooling operations
    """
    net = input_

    with tf.variable_scope("layer{}".format(name)):
        for i, F in enumerate(n_filters):
            net = tf.layers.conv2d(
                net,
                F, (3, 3),
                activation=None,
                padding='same',
                kernel_regularizer=tf.contrib.layers.l2_regularizer(flags["reg"]),
                name="conv_{}".format(i + 1))
            net = tf.layers.batch_normalization(
                net, training=training, name="bn_{}".format(i + 1))
            net = activation(net, name="relu{}_{}".format(name, i + 1))

        if pool is False:
            return net

        pool = tf.layers.max_pooling2d(
            net, (2, 2), strides=(2, 2), name="pool_{}".format(name))

        return net, pool


def upconv_concat(inputA, input_B, n_filter, flags, name):
    """Upsample `inputA` and concat with `input_B`
    Args:
        input_A (4-D Tensor): (N, H, W, C)
        input_B (4-D Tensor): (N, 2*H, 2*H, C2)
        name (str): name of the concat operation
    Returns:
        output (4-D Tensor): (N, 2*H, 2*W, C + C2)
    """
    up_conv = upconv_2D(inputA, n_filter, flags, name)

    return tf.concat(
        [up_conv, input_B], axis=-1, name="concat_{}".format(name))


def upconv_2D(tensor, n_filter, flags, name):
    """Up Convolution `tensor` by 2 times
    Args:
        tensor (4-D Tensor): (N, H, W, C)
        n_filter (int): Filter Size
        name (str): name of upsampling operations
    Returns:
        output (4-D Tensor): (N, 2 * H, 2 * W, C)
    """

    return tf.layers.conv2d_transpose(
        tensor,
        filters=n_filter,
        kernel_size=2,
        strides=2,
        kernel_regularizer=tf.contrib.layers.l2_regularizer(flags["reg"]),
        name="upsample_{}".format(name))


def make_unet(X, training, flags=None):
    """Build a U-Net architecture
    Args:
        X (4-D Tensor): (N, H, W, C)
        training (1-D Tensor): Boolean Tensor is required for batchnormalization layers
    Returns:
        output (4-D Tensor): (N, H, W, C)
            Same shape as the `input` tensor
    Notes:
        U-Net: Convolutional Networks for Biomedical Image Segmentation
        https://arxiv.org/abs/1505.04597
    """
    net = X / 127.5 - 1
    conv1, pool1 = conv_conv_pool(net, [8, 8], training, flags, name=1)
    conv2, pool2 = conv_conv_pool(pool1, [16, 16], training, flags, name=2)
    conv3, pool3 = conv_conv_pool(pool2, [32, 32], training, flags, name=3)
    conv4, pool4 = conv_conv_pool(pool3, [64, 64], training, flags, name=4)
    conv5 = conv_conv_pool(
        pool4, [128, 128], training, flags, name=5, pool=False)

    up6 = upconv_concat(conv5, conv4, 64, flags, name=6)
    conv6 = conv_conv_pool(up6, [64, 64], training, flags, name=6, pool=False)

    up7 = upconv_concat(conv6, conv3, 32, flags, name=7)
    conv7 = conv_conv_pool(up7, [32, 32], training, flags, name=7, pool=False)

    up8 = upconv_concat(conv7, conv2, 16, flags, name=8)
    conv8 = conv_conv_pool(up8, [16, 16], training, flags, name=8, pool=False)

    up9 = upconv_concat(conv8, conv1, 8, flags, name=9)
    conv9 = conv_conv_pool(up9, [8, 8], training, flags, name=9, pool=False)

    return tf.layers.conv2d(
        conv9,
        1, (1, 1),
        name='final',
        activation=tf.nn.sigmoid,
        padding='same')


def IOU_(y_pred, y_true):
    """Returns a (approx) IOU score
    intesection = y_pred.flatten() * y_true.flatten()
    Then, IOU = 2 * intersection / (y_pred.sum() + y_true.sum() + 1e-7) + 1e-7
    Args:
        y_pred (4-D array): (N, H, W, 1)
        y_true (4-D array): (N, H, W, 1)
    Returns:
        float: IOU score
    """
    H, W, _ = y_pred.get_shape().as_list()[1:]

    pred_flat = tf.reshape(y_pred, [-1, H * W])
    true_flat = tf.reshape(y_true, [-1, H * W])

    intersection = 2 * tf.reduce_sum(pred_flat * true_flat, axis=1) + 1e-7
    denominator = tf.reduce_sum(
        pred_flat, axis=1) + tf.reduce_sum(
            true_flat, axis=1) + 1e-7

    return tf.reduce_mean(intersection / denominator)


def make_train_op(y_pred, y_true):
    """Returns a training operation
    Loss function = - IOU(y_pred, y_true)
    IOU is
        (the area of intersection)
        --------------------------
        (the area of two boxes)
    Args:
        y_pred (4-D Tensor): (N, H, W, 1)
        y_true (4-D Tensor): (N, H, W, 1)
    Returns:
        train_op: minimize operation
    """
    loss = -IOU_(y_pred, y_true)

    global_step = tf.train.get_or_create_global_step()

    optim = tf.train.AdamOptimizer()
    return optim.minimize(loss, global_step=global_step)


def read_flags():
    """Returns flags"""
    
    return dict({
        "epochs":100,
        "batch-size":10,
        "logdir":"logdir",
        "reg":0.1,
        "ckdir":"models"
    })
    
    import argparse

    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument(
        "--epochs", default=1, type=int, help="Number of epochs")

    parser.add_argument("--batch-size", default=4, type=int, help="Batch size")

    parser.add_argument(
        "--logdir", default="logdir", help="Tensorboard log directory")

    parser.add_argument(
        "--reg", type=float, default=0.1, help="L2 Regularizer Term")

    parser.add_argument(
        "--ckdir", default="models", help="Checkpoint directory")

    flags = parser.parse_args()
    return flags


def main(flags):
    train = pd.read_csv("augmentation/train1.csv")
    n_train = train.shape[0]

    test = pd.read_csv("augmentation/test1.csv")
    n_test = test.shape[0]

#     current_time = time.strftime("%m/%d/%H/%M/%S")
    train_logdir = os.path.join(flags["logdir"], "train")
    test_logdir = os.path.join(flags["logdir"], "test")

    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape=[None, 768, 768, 3], name="X")
    y = tf.placeholder(tf.float32, shape=[None, 768, 768, 1], name="y")
    mode = tf.placeholder(tf.bool, name="mode")

    pred = make_unet(X, mode, flags)

    tf.add_to_collection("inputs", X)
    tf.add_to_collection("inputs", mode)
    tf.add_to_collection("outputs", pred)

    tf.summary.histogram("Predicted_Mask", pred)
    tf.summary.image("Predicted_Mask", pred)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

    with tf.control_dependencies(update_ops):
        train_op = make_train_op(pred, y)

    IOU_op = IOU_(pred, y)
    IOU_op = tf.Print(IOU_op, [IOU_op])
    tf.summary.scalar("IOU", IOU_op)

#     train_csv = tf.train.string_input_producer(['train.csv'])
#     test_csv = tf.train.string_input_producer(['test.csv'])
    
#     train_image, train_mask = get_image_mask(train_csv)
#     test_image, test_mask = get_image_mask(test_csv, augmentation=False)
    
#     print(train_image)
    
#     return None
    
#     X_batch_op, y_batch_op = tf.train.shuffle_batch(
#         [train_image, train_mask],
#         batch_size=flags["batch_size"],
#         capacity=flags["batch_size"] * 5,
#         min_after_dequeue=flags["batch_size"] * 2,
#         allow_smaller_final_batch=True)

#     X_test_op, y_test_op = tf.train.batch(
#         [test_image, test_mask],
#         batch_size=flags["batch_size"],
#         capacity=flags["batch_size"] * 2,
#         allow_smaller_final_batch=True)

    summary_op = tf.summary.merge_all()
    
    dataset = Dataset(20)
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.8
    
    with tf.Session() as sess:
        train_summary_writer = tf.summary.FileWriter(train_logdir, sess.graph)
        test_summary_writer = tf.summary.FileWriter(test_logdir)

        init = tf.global_variables_initializer()
        sess.run(init)
        
        print("Start training ......")
        
        saver = tf.train.Saver()
        if os.path.exists(flags["ckdir"]) and tf.train.checkpoint_exists(flags["ckdir"]):
            latest_check_point = tf.train.latest_checkpoint(flags["ckdir"])
            print('restoree')
            saver.restore(sess, latest_check_point)

        else:
            try:
                os.rmdir(flags["ckdir"])
            except FileNotFoundError:
                pass
            os.mkdir(flags["ckdir"])

        try:
            global_step = tf.train.get_global_step(sess.graph)

            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(coord=coord)

            for epoch in range(flags["epochs"]):
                print("Epoch ", epoch)
                
                for step in range(dataset.train_batch_count):

                    X_batch, y_batch = dataset.next_aug_train_batch(step)

                    _, step_iou, step_summary, global_step_value = sess.run(
                        [train_op, IOU_op, summary_op, global_step],
                        feed_dict={X: X_batch,
                                   y: y_batch,
                                   mode: True})

                    train_summary_writer.add_summary(step_summary,
                                                     global_step_value)

                total_iou = 0
                for step in range(dataset.test_batch_count):
                    X_test, y_test = dataset.next_test_batch(step)
                    step_iou, step_summary = sess.run(
                        [IOU_op, summary_op],
                        feed_dict={X: X_test,
                                   y: y_test,
                                   mode: False})

                    total_iou += step_iou * X_test.shape[0]

                    test_summary_writer.add_summary(step_summary,
                                                    (epoch + 1) * (step + 1))

            saver.save(sess, "{}/model.ckpt".format(flags["ckdir"]))

        finally:
            coord.request_stop()
            coord.join(threads)
            saver.save(sess, "{}/model.ckpt".format(flags["ckdir"]))

#### main

In [15]:
with tf.device("/device:GPU:0"):
    flags = read_flags()
    main(flags)          

Start training ......
restoree
INFO:tensorflow:Restoring parameters from models/model.ckpt
Epoch  0


ResourceExhaustedError: OOM when allocating tensor with shape[20,8,768,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: gradients/final/Conv2D_grad/Conv2DBackpropInput = Conv2DBackpropInput[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](gradients/final/Conv2D_grad/ShapeN, final/kernel/read, gradients/final/Sigmoid_grad/SigmoidGrad, ^gradients/final/BiasAdd_grad/BiasAddGrad)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'gradients/final/Conv2D_grad/Conv2DBackpropInput', defined at:
  File "/home/ubuntu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_